In [1]:
import torch
import tiktoken

/Users/tinocheek/projects/cs290/venv/lib/python3.9/site-packages/torch/_subclasses/functional_tensor.py:279: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [2]:
raw_text = "This is my dog Raylan"

In [3]:
tokenizer = tiktoken.get_encoding("gpt2")

/Users/tinocheek/projects/cs290/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
enc_text = tokenizer.encode( raw_text )
print(enc_text)

[1212, 318, 616, 3290, 7760, 9620]


In [5]:
vocab_size = 5
output_dim = 8
inputs = torch.nn.Embedding( vocab_size, output_dim )
print( inputs.weight )


Parameter containing:
tensor([[ 0.0760, -1.4988, -0.5494,  1.0273, -1.4077, -1.0539, -0.7224,  0.9394],
        [-0.6874,  1.5714, -0.4658,  0.8110, -0.5534,  0.6439, -0.6450,  0.8761],
        [ 0.2727,  0.4998, -0.3186, -0.6191, -0.8056, -2.0084,  1.4146, -1.0975],
        [-1.4704,  0.2161,  0.5842,  1.4889, -0.5997, -0.3778, -0.9948,  1.0914],
        [-0.3114, -0.4453,  0.0940,  2.5388,  0.5074,  1.1272, -0.8238, -0.4585]],
       requires_grad=True)


In [6]:
inputs = inputs.weight.data
print(inputs)


tensor([[ 0.0760, -1.4988, -0.5494,  1.0273, -1.4077, -1.0539, -0.7224,  0.9394],
        [-0.6874,  1.5714, -0.4658,  0.8110, -0.5534,  0.6439, -0.6450,  0.8761],
        [ 0.2727,  0.4998, -0.3186, -0.6191, -0.8056, -2.0084,  1.4146, -1.0975],
        [-1.4704,  0.2161,  0.5842,  1.4889, -0.5997, -0.3778, -0.9948,  1.0914],
        [-0.3114, -0.4453,  0.0940,  2.5388,  0.5074,  1.1272, -0.8238, -0.4585]])


In [7]:
inputs.shape

torch.Size([5, 8])

In [8]:
for row in inputs:
    print(row.tolist())

[0.07602551579475403, -1.4987521171569824, -0.5493846535682678, 1.0272908210754395, -1.4077391624450684, -1.0539418458938599, -0.7224203944206238, 0.939432680606842]
[-0.687371015548706, 1.57136869430542, -0.4657619595527649, 0.8109579086303711, -0.5533872842788696, 0.6439405083656311, -0.6450309157371521, 0.8760560750961304]
[0.27273932099342346, 0.4997693598270416, -0.318569540977478, -0.6191158890724182, -0.8055837750434875, -2.0083653926849365, 1.4145913124084473, -1.0974879264831543]
[-1.4703824520111084, 0.21611714363098145, 0.5842016339302063, 1.4888867139816284, -0.5996881723403931, -0.377820760011673, -0.9948233366012573, 1.091390609741211]
[-0.3113877773284912, -0.44530269503593445, 0.09400535374879837, 2.538794755935669, 0.5074234008789062, 1.1271592378616333, -0.8238257765769958, -0.4584828019142151]


In [9]:
x = torch.Tensor([1.1,2.3])
y = torch.Tensor([3.4,-2.1])

In [10]:
1.1*3.4 + 2.3*-2.1

-1.0899999999999999

In [11]:
torch.dot(x,y)

tensor(-1.0900)

In [12]:
query = inputs[2]
print(query)

tensor([ 0.2727,  0.4998, -0.3186, -0.6191, -0.8056, -2.0084,  1.4146, -1.0975])


In [13]:
for i in range( len( inputs )):
    print( torch.dot(query, inputs[i]))

tensor(0.0085)
tensor(-2.4772)
tensor(8.6970)
tensor(-2.7641)
tensor(-5.2439)


In [14]:
attention_scores_2 = torch.zeros(len(inputs))
for i in range( len( inputs )):
    attention_scores_2[i] = torch.dot(query, inputs[i])
print(attention_scores_2)

tensor([ 8.5166e-03, -2.4772e+00,  8.6970e+00, -2.7641e+00, -5.2439e+00])


In [15]:
# #normalize the attention scores using the softmax function
# def softmax(x):
#     torch.exp(x) / torch.exp(x).sum()

In [16]:
attention_weights_2 = torch.softmax(attention_scores_2, dim = 0)
print(attention_weights_2)
print( attention_weights_2.sum() )

tensor([1.6848e-04, 1.4028e-05, 9.9981e-01, 1.0530e-05, 8.8195e-07])
tensor(1.)


In [17]:
context_vector_2 = torch.zeros( query.shape )
for i in range(len(attention_weights_2)):
    context_vector_2 += attention_weights_2[i]*inputs[i]
print(context_vector_2)

tensor([ 0.2727,  0.4994, -0.3186, -0.6188, -0.8057, -2.0081,  1.4142, -1.0971])


In [18]:
# get all of the attention scores by matrix multiplication
attention_scores = inputs @ inputs.T
attention_scores

tensor([[ 8.1061e+00,  7.0951e-02,  8.5165e-03,  3.7593e+00,  1.4623e+00],
        [ 7.0951e-02,  5.7207e+00, -2.4772e+00,  3.9720e+00,  2.1041e+00],
        [ 8.5165e-03, -2.4772e+00,  8.6970e+00, -2.7641e+00, -5.2439e+00],
        [ 3.7593e+00,  3.9720e+00, -2.7641e+00,  7.4500e+00,  3.7855e+00],
        [ 1.4623e+00,  2.1041e+00, -5.2439e+00,  3.7855e+00,  9.1664e+00]])

In [19]:
attention_weights = torch.softmax( attention_scores, dim = -1)
attention_weights

tensor([[9.8534e-01, 3.1912e-04, 2.9980e-04, 1.2757e-02, 1.2830e-03],
        [2.9206e-03, 8.3010e-01, 2.2846e-04, 1.4444e-01, 2.2309e-02],
        [1.6848e-04, 1.4028e-05, 9.9981e-01, 1.0530e-05, 8.8195e-07],
        [2.3074e-02, 2.8544e-02, 3.3890e-05, 9.2466e-01, 2.3688e-02],
        [4.4832e-04, 8.5177e-04, 5.4839e-07, 4.5766e-03, 9.9412e-01]])

In [20]:
attention_weights.sum(axis = -1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000])

In [21]:
context_vectors = attention_weights @ inputs
context_vectors

tensor([[ 0.0556, -1.4739, -0.5340,  1.0346, -1.3945, -1.0423, -0.7254,  0.9389],
        [-0.7896,  1.3214, -0.3018,  0.9477, -0.5390,  0.5016, -0.6993,  0.8771],
        [ 0.2727,  0.4994, -0.3186, -0.6188, -0.8057, -2.0081,  1.4142, -1.0971],
        [-1.3848,  0.1996,  0.5164,  1.4837, -0.5908, -0.3287, -0.9744,  1.0450],
        [-0.3168, -0.4410,  0.0955,  2.5318,  0.5006,  1.1189, -0.8244, -0.4496]])